<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/Real_state_url_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re,os
import time
from datetime import datetime
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
for dirname, _, filenames in os.walk('/content/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [17]:
def get_data(url):  
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    real_estate_data = soup.find_all("dl", class_='dl-horizontal-border')
    is_property_found = "Yes"
    data_dict = {}
    data_dict["URL"] = url
    if real_estate_data:
        for d in real_estate_data:
            dt = d.find_all('dt')
            dd = d.find_all('dd')
            
            for i,j in zip(dt, dd):
                i = i.contents[0].strip()
                j = j.contents[0].strip()
                if i == "Unit Number":
                    if j == "-":
                        data_dict["Unit Number"] = np.nan
                    else:
                        data_dict["Unit Number"] = j
                elif i == "Price":
                    data_dict["Price(¥)"] = float(j.replace(",","").replace("¥","").strip())
                elif i == "Building Name":
                    data_dict["Building Name"] = j
                elif i == "Available From":
                    if "Please Inquire" in j:
                        data_dict["Available From"] = np.nan
                    else:
                        data_dict["Available From"] = datetime.strptime(j, '%b %d, %Y')
                elif i == "Type":
                    data_dict["Type"] = j.replace(" ", "")
                elif i == "Size":
                    data_dict["Size(m²)"] = float(j.replace("m²", "").replace(",", "").strip())
                elif i == "Gross Yield":
                    data_dict["Gross Yield(%)"] = float(j.replace("%", "").strip())
                elif i == "Land Rights":
                    data_dict["Land Rights"] = j
                elif i == "Maintenance Fee":
                    data_dict["Maintenance Fee(¥/mnt)"] = float(j.replace("¥", "").replace(" / mth", "").strip().replace(",",""))
                elif i == "Location":
                    data_dict["Location"] = j.replace(",", "")
                elif i == "Occupancy":
                    data_dict["Occupancy"] = j
                elif i == "Floor":
                    data_dict["Floor"] = j.replace(" ", "")
                elif i == "Nearest Station":
                    data_dict["Nearest Station"] = j.split("(")[0].strip()
                    if len(j.split("(")) > 1:
                        if "walk" in j:
                            data_dict["Way to Nearest Station"] = "Walk"
                            data_dict["Distance From Station(min)"] = j.split("(")[1].split("min")[0].strip()
                        elif "bus" in j:
                            data_dict["Way to Nearest Station"] = "Bus"
                            data_dict["Distance From Station(min)"] = j.split("(")[1].split("min")[0].strip()
                elif i == "Layout":
                    data_dict["Layout"] = j
                elif i == "Year Built":
                    data_dict["Year Built"] = j
                elif i == "Direction Facing":
                    data_dict["Direction Facing"] = j.replace(",", "")
                elif i == "Transaction Type":
                    data_dict["Transaction Type"] = j
                elif i == "Balcony Size":
                    data_dict["Balcony Size(m²)"] = float(j.replace("m²", "").replace(",", "").strip())
                elif i == "Building Description":
                    data_dict["Building Description"] = j.replace(",", "")
                elif i == "Other Expenses":
                    j = j.replace(",", "").replace(" ", "").replace("，", "")
                    lst = re.findall(r'\d+', j)
                    if len(lst) > 0:
                        lst = [int(i) for i in lst] 
                        data_dict["Other Expenses"] = sum(lst)
                elif i == "Parking":
                    data_dict["Parking Available"] = j.split()[0].replace(",", "")
                    if len(j.split()) > 1:
                        if j.split()[0].replace(",", "") == "Available":
                            data_dict["Parking Fee(¥/mnt)"] = float(j.split()[1].replace(",", "").replace("¥", "").strip())
                elif i == "Date Updated":
                    if "Please Inquire" in j:
                        data_dict["Date Updated"] = np.nan
                    else:
                        data_dict["Date Updated"] = datetime.strptime(j, '%b %d, %Y')
                elif i == "Next Update Schedule":
                    if "Please Inquire" in j:
                        data_dict["Next Update Schedule"] = np.nan
                    else:
                        data_dict["Next Update Schedule"] = datetime.strptime(j, '%b %d, %Y')
                    
    else:
        is_property_found = "No"
    data_dict["Is_Prop_Avl"] = is_property_found
    return data_dict

## Data Cleaning

In [22]:
!pip install cssselect

In [24]:
from urllib.parse import urlencode, urlparse, parse_qs

from lxml.html import fromstring
from requests import get

raw = get("https://www.google.com/search?q=StackOverflow").text
page = fromstring(raw)

for result in page.cssselect(".r a"):
    url = result.get("href")
    if url.startswith("/url?"):
        url = parse_qs(urlparse(url).query)['q']
    print(url[0])

In [28]:
!pip install google-search-results


  Created wheel for google-search-results: filename=google_search_results-2.4.0-py3-none-any.whl size=19876 sha256=946b304bfacaa40e2b364f4e925bcba082545180d76fa81e10b3aabb01b490a7
  Stored in directory: /root/.cache/pip/wheels/93/32/5f/313cc6b6c013f9c6386fc0a75d10b6b1aa7fe6867acc704d41
Successfully built google-search-results


In [ ]:
!pip install search-engine-parser

In [3]:
from search_engine_parser import GoogleSearch

def google(query):
    search_args = (query, 1)
    gsearch = GoogleSearch()
    gresults = gsearch.search(*search_args)
    return gresults['links']

google('Is it illegal to scrape google results')

RuntimeError: ignored

In [6]:
!virtualenv --python python3 env
!source env/bin/activate
!pip install git+git://github.com/NikolaiT/GoogleScraper/

/bin/bash: virtualenv: command not found
/bin/bash: env/bin/activate: No such file or directory
  Cloning git://github.com/NikolaiT/GoogleScraper/ to /tmp/pip-req-build-2me4yc3z
  Running command git clone -q git://github.com/NikolaiT/GoogleScraper/ /tmp/pip-req-build-2me4yc3z


In [7]:
# !GoogleScraper -m http --keyword SearchData/some_words.txt --num-workers 5 --search-engines "bing,yahoo" --output-filename threaded-results.json -v debug
# !GoogleScraper -m http --keyword "apple" -v info
!GoogleScraper -h


Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sqlalchemy/sql/base.py", line 1205, in __getattr__
    return self._index[key]
KeyError: '_data'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/bin/GoogleScraper", line 5, in <module>
    from GoogleScraper.core import main
  File "/usr/local/lib/python3.7/dist-packages/GoogleScraper/__init__.py", line 20, in <module>
    from GoogleScraper.core import scrape_with_config
  File "/usr/local/lib/python3.7/dist-packages/GoogleScraper/core.py", line 14, in <module>
    from GoogleScraper.caching import CacheManager
  File "/usr/local/lib/python3.7/dist-packages/GoogleScraper/caching.py", line 12, in <module>
    from GoogleScraper.output_converter import store_serp_result
  File "/usr/local/lib/python3.7/dist-packages/GoogleScraper/output_converter.py", line 19, in <module>
    csv_fieldnames = sorted(set(Link.__table__.columns._dat

In [ ]:
!pip install git+https://github.com/abenassi/Google-Search-API

In [14]:
from googleapi import google
num_page = 1
search_results = google.search("This ", num_page)
print(search_results)

[]


In [15]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.google.com/search?q=املاک+تهران+اجاره"
r = requests.get(url)
page = r.text 
soup = BeautifulSoup(page, 'lxml') 

i = 0

link_list = []
for tag in soup.find_all('a'):
    i+=1
    href = tag['href']
    if re.search('http',href):
        try:
            link = re.search('https://.+\.com',href).group(0)
            link_list.append(link)
        except:
            pass

link_list = list(set(link_list))

link_list2 = [] 

for link in link_list:
    if not re.search('google.com',link):
        link_list2.append(link)
        
print(link_list2)

# import pandas as pd

# df = pd.DataFrame(link_list2)
# df.to_csv('/content/sample_data/bs4_final.csv', index=False)

[]


In [19]:
df = pd.read_csv("/content/sample_data/bs4_final.csv" , delim_whitespace=True)#"/kaggle/input/japanese-property-urls/tokyo_property_urls.csv")
print(df)

EmptyDataError: ignored

In [ ]:
urls = df.columns[0:]
len(urls)
print (urls)

## Scrape Data

In [ ]:
real_estate_df = pd.DataFrame(columns=["URL", "مناسب برای", "شماره واحد", "قیمت", "نام ساختمان", "طبقه", "در دسترس از", "نوع", "اندازه", "بازده ناخالص",
                                      "حقوق زمین", "هزینه نگهداری", "موقعیت مکانی", "شغل", "نزدیکترین ایستگاه", "راه تا نزدیکترین ایستگاه", "فاصله از ایستگاه (دقیقه",
                                      "نما", "سال ساخت", "رو به جهت", "نوع معامله", "اندازه بالکن (متر مربع)", "شرح ساختمان", "سایر هزینه ها",
                                      "پارکینگ موجود", "هزینه پارکینگ", "برنامه به‌روزرسانی بعدی", "برنامه به‌روزرسانی بعدی"])
for url in urls:
    print (url)
    # res = get_data(url)
    # real_estate_df = real_estate_df.append(res, ignore_index=True)

In [2]:
%cd '/kaggle/input/d/sotest/crawler/'
import ipykernel_launcher
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import colorama

# init the colorama module
colorama.init()

GREEN = colorama.Fore.GREEN
GRAY = colorama.Fore.LIGHTBLACK_EX
RESET = colorama.Fore.RESET
YELLOW = colorama.Fore.YELLOW

# initialize the set of links (unique links)
internal_urls = set()
external_urls = set()

total_urls_visited = 0


def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_all_website_links(url):
    """
    Returns all URLs that is found on `url` in which it belongs to the same website
    """
    # all URLs of `url`
    urls = set()
    # domain name of the URL without the protocol
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parser = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                print(f"{GRAY}[!] External link: {href}{RESET}")
                external_urls.add(href)
            continue
        print(f"{GREEN}[*] Internal link: {href}{RESET}")
        urls.add(href)
        internal_urls.add(href)
    return urls


def crawl(url, max_urls=30):
    """
    Crawls a web page and extracts all links.
    You'll find all links in `external_urls` and `internal_urls` global set variables.
    params:
        max_urls (int): number of max urls to crawl, default is 30.
    """
    global total_urls_visited
    total_urls_visited += 1
    print(f"{YELLOW}[*] Crawling: {url}{RESET}")
    links = get_all_website_links(url)
    for link in links:
        if total_urls_visited > max_urls:
            break
        crawl(link, max_urls=max_urls)


# if __name__ == "__main__":
if True:
#     import argparse
#     parser = argparse.ArgumentParser(description="Link Extractor Tool with Python")
#     parser.add_argument("url", help="The URL to extract links from.")
#     parser.add_argument("-m", "--max-urls", help="Number of max URLs to crawl, default is 30.", default=30, type=int)
    
#     args = parser.parse_args()
    url = "https://short.gy/d3RXQ9"#args.url
    url = "https://www.zoomila.com"
    max_urls = '1'#args.max_urls

    crawl(url, max_urls=max_urls)

    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total External links:", len(external_urls))
    print("[+] Total URLs:", len(external_urls) + len(internal_urls))
    print("[+] Total crawled URLs:", max_urls)

    domain_name = urlparse(url).netloc

    # save the internal links to a file
    with open(f"{domain_name}_internal_links.txt", "w") as f:
        for internal_link in internal_urls:
            print(internal_link.strip(), file=f)

    # save the external links to a file
    with open(f"{domain_name}_external_links.txt", "w") as f:
        for external_link in external_urls:
            print(external_link.strip(), file=f)

[Errno 2] No such file or directory: '/kaggle/input/d/sotest/crawler/'
/content
[*] Crawling: https://www.zoomila.com


NameError: ignored